In [93]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD! https://stackoverflow.com/questions/31984196/ipython-notebook-run-all-cells-on-open
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
               //IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
           // $('#header-container').hide();
          //$('#maintoolbar').hide();   
        }
    );

</script>

In [94]:
from IPython.display import HTML, clear_output,Javascript, display
from ipywidgets import widgets
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Box, VBox, HBox, Label,FloatProgress
import requests
import json
import pandas as pd
from pandas.compat import StringIO, BytesIO
from pprintpp import pprint as pp
from pandas.io.json import json_normalize
import time
import datetime as dt
import pytz
import base64
import qgrid
import io 
import fileupload

pd.set_option("max_r", 102)
pd.set_option('display.max_colwidth', -1)

In [95]:

#pd.set_option('display.max_rows', 999)
# I wish i knew how to always hide this cell !

# button[data-event-type$=_row] {
#    display: none !important;}

# button.btn.btn-default.fa.fa-arrows-alt.full-screen-btn { margin-left:5px;}


HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
<style>
button[data-event-type$=_row] {
    display: none !important;}

 button.btn.btn-default.fa.fa-arrows-alt.full-screen-btn { margin-left:5px;}
</style>''')


# Welcome 👋
### Purpose
Configure Course sites that are using Barnes and Noble LTI at scale. 


### ⚡️ Quick Directions ⚡️

1. enter in api token ( don't know what that is? please see: https://community.canvaslms.com/docs/DOC-10806 )
3. select canvas environment ( currently only production is set )
4. click submit ( make sure the top turns green )
5. scroll down to see further directions

**note**: if things seem bizarre refresh the page and the go to Cell > Run All in the top menu


In [96]:
error = widgets.Output()

API_widget = widgets.Text(placeholder='API TOKEN',
                       layout=Layout(width='90%',align_items='center', grid_area='main'))


Authentication = Button(
    value=False,
    description='NOT Authorized',
    disabled=True,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='unlock',
    layout=Layout(width='auto', grid_area='header')
)


Submit = Button(
    value=False,
    description='Submit',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',
    layout=Layout(width='auto', grid_area='footer')
)



# for the env // Domain
environ = widgets.Select(
    options=['Production', 'Test', 'Beta'],
    value='Production',
    description='Environment:',
    disabled=False,
    rows=3,
    layout=Layout(width='auto', grid_area='sidebar')
)

environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'https://upenn.test.instructure.com/api/v1/', 'Beta':'https://upenn.beta.instructure.com/api/v1/'}



def set_up_forms():
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+6)'))

def authorize():
    # check api key 
    global headers
    headers = {
        'Authorization': 'Bearer %s' % (API_widget.value) ,
    }
    global domain
    domain = environs[environ.value]
    print("domain", domain)
    
    x = test_api() # returns status code of test response
    
    if API_widget.value and x ==200:
    # if good:
        
        # UI changes
        Authentication.icon = 'lock'
        Authentication.description = 'Authorized'
        Authentication.button_style ='success'
        set_up_forms()
    #Authentication.style.button_color = 'lightgreen'
    # if bad:
    else:
        error.append_stdout("Invalid API token")
        Authentication.description='NOT Authorized'
        Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
        Authentication.icon='unlock'
        headers = {}
    # output note 
    


def on_button_clicked(b):
    # clear any output ( error output from authorize )
    error.clear_output()
    authorize()
    
    
Submit.on_click(on_button_clicked)


#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Grid-layout

header  = Authentication
main    = API_widget
sidebar = environ
footer  = Submit



form = GridBox(children=[header, main, sidebar,footer],
        layout=Layout(
            width='80%',
            margin="2% 0% 2% 10%",
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            #grid_gap='5px 5px',
            grid_template_areas='''
            " . header header ."
            "main main sidebar sidebar"
            " . footer footer . "
            ''')
       )

display(HTML("<a name='apiForm'></a><h2 style='text-align: center;' >Please enter your API Credentials</h2>"))
display(form)
display(error)


def on_value_change(change):
    Authentication.description='NOT Authorized'
    Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
    Authentication.icon='unlock'
    headers = {'Authorization': 'Bearer %s' % (change['new']) ,}
    
API_widget.observe(on_value_change, names='value')


domain = environs[environ.value]
headers = {}
def test_api():
    #headers = {
    #'Authorization': 'Bearer %s' % (API_widget.value) ,
    #}
    url = domain + 'accounts/self/scopes'
    response = requests.get(url,headers=headers)
    r = response.json()
    #print(response.status_code)
    return response.status_code



######################################
######### HELPER FUNCTION ############
######################################

def srs_format(s):
    #
    numbers = ''.join([n for n in s if n.isdigit()]).replace(" ", "")
    dept = ''.join([n for n in s if n.isdigit()==False]).replace(" ", "")
    level = numbers[:3]
    section = numbers[3:]
    return('SRS_%s-%s-%s' % (dept,level,section))
def create_download_link( df, filename):  
    title = "Download CSV file"
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

GridBox(children=(Button(button_style='warning', description='NOT Authorized', disabled=True, icon='unlock', l…

Output()

domain https://upenn.test.instructure.com/api/v1/


<IPython.core.display.Javascript object>

# Barnes and noble integration 
upload a file and then run a script that reads the file and configures the Barnes and Noble LTI for 

[Barnes and Noble LTI Guide](https://www.facultyenlight.com/sites/default/files/u53/CanvasAdminGuide.pdf)

### Steps
1. Select Term 
2. Upload file 
   file requirements 
   - _must_ be a CSV
   - _must_ have a column with header `Course ID` which contains **valid** course codes like 'COMM210301'

#### <center>Example Upload File</center>
| Course ID  | Instructor          | Email                      | School |
|------------|---------------------|----------------------------|--------|
| COMM210301 | xxxxxxxx,xxx        | xxxxx.upenn.edu            | AN     |
| COMM211301 | xxxxxx,xxxxxx       | xxxxxxxxx.upenn.edu        | AN     |
| COMM225001 | xxxxxx,xxxxxx       | xxxxxxxxx.upenn.edu        | AN     |


In [102]:
output_df = widgets.Output()
messages = widgets.Output()
input_term = widgets.Select(
    options=['2019A', '2019B', '2019C'],
    value='2019A',
    description='Term',
    disabled=False,
    rows=3
)

_upload_widget = fileupload.FileUploadWidget()

def _cb(change):
    output_df.clear_output(wait=True)
    messages.clear_output(wait=True)
    decoded = io.StringIO(change['owner'].data.decode('utf-8'))
    filename = change['owner'].filename
    with messages:
        display(HTML('<p> Uploaded `{}` ({:.2f} kB)</p>'.format(
        filename, len(decoded.read()) / 2 **10)))
    
    #creates dataframe
    df = pd.read_csv(BytesIO(_upload_widget.data))
    #formatting
    term = input_term.value
    df['srs_id'] = df['Course ID'].apply(srs_format)+' '+term
    #display(df)

    with output_df:
        display(HTML('<center><h3>TABLE</h3></center>'))
        display(df)

form = HBox([input_term,_upload_widget],layout=Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    #align_items='stretch',
                    justify_content='space-between',
                    width='50%'),)
display(HTML('<hr><h2>Select term and add file</h2>'))
display(form)
_upload_widget.observe(_cb, names='data')

#SRS_COMM-211-301 2014C


In [103]:
display(messages)
if headers == {}:
    display(HTML("<p style='text-align: center;'>No API Token set</p>"))
    
else:
    display(output_df)

Output()

Output()

In [104]:

pd.set_option('display.max_colwidth', -1)

output_run = widgets.Output()
output_errors = widgets.Output()
f = FloatProgress()

run_button = Button(  
    value=False,
    description='Add Tool To Courses',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',   
    layout=Layout(justify_content='center')
)

def add_tool(srs_id,tool_id):
    #
    #tool_id = 'context_external_tool_121790'
    data = {
        'hidden': 'false',
        'position': '3'
    }
    url = domain + 'courses/sis_course_id:'+srs_id +'/tabs/'+tool_id 
    response = requests.put(url, headers=headers, data=data)
    return response.status_code

def config_course(srs_id):
    #/api/v1/courses/sis_course_id:A1234/assignments
    # https://canvas.upenn.edu/courses/sis_course_id:SRS_COMM-211-301%202014C/
    url = domain + 'courses/sis_course_id:'+srs_id +'/'
    response = requests.get(url,headers=headers)
    if response.status_code != 200: 
        return({'name':srs_id,'course_status':str(response.status_code),'tool_status':'NA'})
    r = response.json()
    #print(r)
    tool_id = 'context_external_tool_160657'
    tool_status = add_tool(srs_id,tool_id)
    return({'name':r['name'],'course_status':str(url).replace("/api/v1",""), 'tool_status':str(tool_status)})


def config_():

    #creates dataframe
    df = pd.read_csv(BytesIO(_upload_widget.data))
    #formatting
    term = input_term.value
    df['srs_id'] = df['Course ID'].apply(srs_format)+' '+term

    courses = df['srs_id']
    count = df.shape[0] # gets the rows
    #create loading bar
    f.min=0
    f.max=count
    f.description='(0/%s)' % (count)
    
    #print(count, ' total course sites')
    counter = 0
    rows_list = []
    for course in courses:
        counter +=1
        f.value += 1
        f.description ='(%s/%s)' % (counter,count)
        dict1 = config_course(course)
        prog = '(%s/%s)' % (counter,count)
        #print(prog, dict1)
        rows_list.append(dict1)
    pd.set_option('display.max_rows', None)
    df = pd.DataFrame(rows_list)   
    with output_run:
        #qgrid.show_grid(common_filter, show_toolbar=True, precision=None, grid_options={'editable': False})
        display(df)
        

def on_button_clicked(b):
    output_run.clear_output(wait=True)
    output_errors.clear_output(wait=True)
    if headers !={}:
    
        run_button.button_style = 'info'
        run_button.description = 'Running...'
        
        config_()
        run_button.button_style = 'success'
        run_button.description = 'Completed!'
    else:
        with output_errors:
            display(HTML("<p> Cannot Run Unless Authenticated please validate <a href='#apiForm'>here</a></p>"))
   

display(run_button)
display(f)
display(output_errors)
run_button.on_click(on_button_clicked)

#SRS_COMM-211-301 2014C
#print(config_course('SRS_COMM-211-301 2014C'))


#### Tab example 
#{"id":"context_external_tool_160657","html_url":"/courses/1449513/external_tools/160657",
# "full_url":"https://upenn.test.instructure.com/courses/1449513/external_tools/160657","position":25,
# "hidden":true,"visibility":"admins","label":"Purchase Course Materials","type":"external",
# "url":"https://upenn.test.instructure.com/api/v1/courses/1449513/external_tools/sessionless_launch?id=160657\u0026launch_type=course_navigation"}

Button(button_style='info', description='Add Tool To Courses', layout=Layout(justify_content='center'), style=…

FloatProgress(value=0.0)

Output()

In [105]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', -1)
link = widgets.Output()




def on_download_clicked(b):
    #download to csv
    link.clear_output()
    #print(pageview_output.capture())
    if output_run.outputs:
        #print("ok")
        #print("csv_data.outputs",csv_data.outputs)
                
        content = output_run.outputs[0]['data']['text/html']
        #print(content)
        
        data = pd.read_html(content)[0]
        data.drop('Unnamed: 0', axis=1, inplace=True)
        #display(data)
        csv_name = filename.value
        with link:
            display(create_download_link(data,csv_name))
    else:
        with link:
            display(HTML('<p>no content selected</p>'))


download = Button(
    value=False,
    description='Download Results',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='fa-download',)
filename = widgets.Text(value='BN_LTI_config.csv',description='filename')
accordion = widgets.Accordion(children=[widgets.HBox([filename,download,link])])
accordion.set_title(0, 'Download')
accordion.selected_index = None


download.on_click(on_download_clicked)

if headers == {}:
    display(HTML("<p style='text-align: center;'>No API Token set</p>"))
else:
    #display(HTML("<style>.rendered_html th {max-width: 120px;}</style>"))
    display(HTML("<h3>Results</h3>"))
    display(output_run)
    display(accordion)

Output()

Accordion(children=(HBox(children=(Text(value='BN_LTI_config.csv', description='filename'), Button(button_styl…

,course_status,name,tool_status
0,404,SRS_ACCT-102-920 2019B,NaN
1,404,SRS_ANTH-003-920 2019B,NaN
2,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIBB-217-920 2019B/,BIBB 217 2019B Visual Neuroscience,200.0
3,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIOL-101-920 2019B/,BIOL 101 2019B Intro Biology A,200.0
4,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIOL-102-920 2019B/,BIOL 102 2019B Intro Biology B,200.0
5,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIOL-201-900 2019B/,BIOL 201 2019B Essentials Of Cell Bio,200.0
6,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIOL-203-900 2019B/,BIOL 203 2019B Essentials Of Biochem,200.0
7,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIOL-404-900 2019B/,BIOL 404 2019B IMMUNOBIOLOGY,200.0
8,https://upenn.test.instructure.com/courses/sis_course_id:SRS_BIOL-407-900 2019B/,BIOL 407 2019B Cancer Cell Biology,200.0
9,404,SRS_CHEM-102-920 2019B,NaN


<hr>

##### my notes
create the external tool 
https://canvas.instructure.com/doc/api/external_tools.html#method.external_tools.create  
 
launch url: https://blti.bncollege.com/lti/1.1/launch/bn_cmp  

Canvas API Docs  
https://canvas.instructure.com/doc/api/tabs.html#method.tabs.index  
https://canvas.instructure.com/doc/api/external_tools.html  


https://canvas.instructure.com/doc/api/file.object_ids.html  
`/api/v1/courses/sis_course_id:A1234/assignments`

In [101]:
# this cell was taken from https://gist.github.com/nokados/e8f0a64b55099f2f07a50f2b090c91c7

from IPython.core.display import display, HTML, clear_output
from ipywidgets import widgets, Layout
import math
PAGESIZE=10

class Paginator:
    def __init__(self, df, pagesize = PAGESIZE, start_page = 0):
        self.df = df
        self.pagesize = pagesize
        self.page = start_page
        self.max_pages = math.ceil(df.shape[0] / self.pagesize)
        self.btn_next = widgets.Button(description='Next')
        self.btn_next.on_click(self.next_page)
        self.btn_prev = widgets.Button(description='Prev')
        self.btn_prev.on_click(self.prev_page)
        

    def get_page(self):
        return self.df.iloc[self.page * self.pagesize: (self.page + 1) * self.pagesize]

    def show(self):
        clear_output()
        self.control = widgets.HBox((self.btn_prev, 
                                     widgets.Label('PAGE {}/{}'.format(self.page + 1, self.max_pages)),
                                     self.btn_next))
        display(self.control)
        display(self.get_page())
        display(self.control)
    
    def next_page(self, b):
        self.page = min(self.max_pages - 1, self.page + 1)
        self.show()
        
    def prev_page(self, b):
        self.page = max(0, self.page - 1)
        self.show()